# CSV file with test set

Import necessary libraries

In [1]:
import os
import glob
import pandas as pd

In [2]:
cwd = os.getcwd()
parent = os.path.dirname(cwd)
parent

'/home/niko/Documents/Python/reynoldsDeepL'

In [3]:
case_dir = os.path.join(parent, "flowCases/")
case_list = os.listdir(case_dir)
case_list = [case for case in case_list if 'test' in case]
case_list

['testCaseRe=10.50',
 'testCaseRe=20.00',
 'testCaseRe=1.00',
 'testCaseRe=5.75',
 'testCaseRe=15.25']

Read a value from a sample case

In [4]:
case_path = os.path.join(case_dir, case_list[0])
postFile = glob.glob(f"{case_path}/postProcessing/*/0/surfaceFieldValue.dat")[0]
postFile

'/home/niko/Documents/Python/reynoldsDeepL/flowCases/testCaseRe=10.50/postProcessing/patchAverage(name=inlet,p)/0/surfaceFieldValue.dat'

In [5]:
with open(postFile) as f:
    lines = f.readlines()
    print(float(lines[-1].split()[1]))  # last time, second column

8.73004


Let's turn what we did into a script for all of the cases and create the master CSV file

In [6]:
# Function to read value from file
def read_value(path):
        try:
            with open(path) as f:
                lines = f.readlines()
                return float(lines[-1].split()[1])  # last time, second column
        except:
            return None

In [7]:
import re
pattern = r'\d+\.\d{2}'

In [8]:
cwd = os.getcwd()
parent = os.path.dirname(cwd)
base_dir = os.path.join(parent, "flowCases/")
case_list = os.listdir(base_dir)
case_list = [case for case in case_list if 'test' in case]
rows = []

for case in case_list:
    case_dir = os.path.join(base_dir, case)
    if not os.path.isdir(case_dir):
        continue
    p_in = read_value(glob.glob(f"{case_dir}/postProcessing/*/0/surfaceFieldValue.dat")[0])
    # Create dictionary for each case
    rows.append({
        "p_in": p_in,
        "Re": re.search(pattern, case).group(0) if re.search(pattern, case) else None,
        "Geometry": case.split('=')[0] if '=' in case else None        
    })

In [10]:
df = pd.DataFrame(rows)
df.head(5)

,p_in,Re,Geometry
0,8.730040,10.50,testCaseRe
1,5.878845,20.00,testCaseRe
2,75.122210,1.00,testCaseRe
3,14.111010,5.75,testCaseRe
4,6.840435,15.25,testCaseRe


In [11]:
csv_folder = os.path.join(parent, "data")
if not os.path.exists(csv_folder):
    os.makedirs(csv_folder)
csv_path = os.path.join(csv_folder, "test_set.csv")
df.to_csv(csv_path, index=False)